In [49]:
#pip install yfinance
#pip install PyPortfolioOpt

In [2]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import pypfopt
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

### Engine

In [38]:
def Engine(stock_list, START, END, variations):

    sector_mapper, sector_lower, sector_upper = variations
    df = pd.DataFrame()
    for company in stock_list:
        stock = yf.Ticker(company)
        hist = stock.history(start=START, end=END).Close
        df[company] = hist
    df = df.dropna()

    if sector_mapper and sector_lower and sector_upper:
        stock_weights, cleaned_weights = get_weights_variation(df,sector_mapper,sector_lower,sector_upper)
    else:
        stock_weights, cleaned_weights = get_weights(df)
    log_returns = np.log(df/df.shift(1))
    log_returns_cov = log_returns.cov()

    print(cleaned_weights)
    weight_method(stock_weights,log_returns,log_returns_cov)

    share_method(df, cleaned_weights)

    return None

def calculate_returns(weights, returns):
    return np.sum(returns.mean() * weights) * 252

def calculate_volatility(weights, returns_covariance):
    cov_dot_weights = np.dot(returns_covariance * 252, weights)
    variance = np.dot(weights, cov_dot_weights)
    return np.sqrt(variance)

def get_weights(df):
    mu = mean_historical_return(df)
    S = CovarianceShrinkage(df).ledoit_wolf()

    ef = EfficientFrontier(mu, S)

    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()

    ef.portfolio_performance(verbose=True)

    stock_weights = list(cleaned_weights.values())
    
    return stock_weights, cleaned_weights

def get_weights_variation(df,sector_mapper,sector_lower,sector_upper):
    mu = mean_historical_return(df)
    S = CovarianceShrinkage(df).ledoit_wolf()

    ef = EfficientFrontier(mu, S)

    ef.add_sector_constraints(sector_mapper, sector_lower=sector_lower, sector_upper=sector_upper)

    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()

    ef.portfolio_performance(verbose=True)

    stock_weights = list(cleaned_weights.values())
    
    return stock_weights, cleaned_weights

def weight_method(stock_weights,log_returns,log_returns_cov):
    risk_free_rate = 0

    # Calculate the expected portfolio return
    log_portfolio_return = calculate_returns(stock_weights, log_returns)
    portfolio_return = np.exp(log_portfolio_return)-1

    # Calculate the portfolio volatility
    log_portfolio_volatility = calculate_volatility(stock_weights, log_returns_cov)
    portfolio_volatility = np.exp(log_portfolio_volatility) - 1

    # Calculate the Sharpe ratio
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility

    print("----------------------Weight method----------------------")
    print("Portfolio Return:", portfolio_return)
    print("Portfolio Volatility:", portfolio_volatility)
    print("Sharpe Ratio:", sharpe_ratio)

def share_method(df,cleaned_weights):

    risk_free_rate = 0
    portfolio_value = 1000000

    # Calculate the number of shares to purchase using DiscreteAllocation
    #latest_prices = get_latest_prices(df)
    earliest_prices = df.iloc[0]

    da = DiscreteAllocation(cleaned_weights, earliest_prices, total_portfolio_value=portfolio_value)
    #allocation, leftover = da.lp_portfolio(solver="CBC")
    allocation, leftover = da.greedy_portfolio()

    all_tickers = df.columns
    quantity_to_buy = [allocation.get(ticker, 0) for ticker in all_tickers]

    # Calculate the daily portfolio value
    daily_portfolio_value = np.dot(df, quantity_to_buy)
    daily_portfolio_value += leftover
    df_portfolio_value = pd.DataFrame(data={'Portfolio Value': daily_portfolio_value}, index=df.index)
    portfolio_log_returns = np.log(df_portfolio_value/df_portfolio_value.shift(1))

    # Annual Returns
    portfolio_annual_returns = portfolio_log_returns.mean() * 252
    annual_regular_ret = np.exp(portfolio_annual_returns)-1


    # Standard Deviation
    daily_portfolio_sd = (np.exp(portfolio_log_returns)-1).std()
    annual_std =  daily_portfolio_sd * (252 **0.5)

    # Sharpe Ratio
    sharpe_ratio_discrete = (annual_regular_ret - risk_free_rate) / annual_std

    print("----------------------Share method----------------------")
    print("Total Portfolio Return with Discrete Allocation:", annual_regular_ret)
    print("Portfolio Volatility with Discrete Allocation:", annual_std)
    print("Sharpe Ratio with Discrete Allocation:", sharpe_ratio_discrete)


### Stock Selection

In [15]:
# Stock Selection
stock_list = ["Z74.SI","D05.SI","S68.SI","C38U.SI","S58.SI","S08.SI","C6L.SI","CC3.SI","S63.SI","BN4.SI","S51.SI","Z59.SI","O39.SI","C09.SI","F99.SI"]
#Singtel, DBS, SGX, Capitaland, SATS, Singapore Post, SIA, Starhub, ST eng, Keppel coporation, Sembcorp, yoma, ocbc, city developments


### Variations

In [36]:
sector_mapper_1 = {
    "Z74.SI": "Communication",
    "D05.SI": "Finance",
    "S68.SI": "Finance",
    "C38U.SI": "Real Estate",
    "S58.SI": "Industrials",
    "S08.SI": "Industrials",
    "C6L.SI": "Industrials",
    "CC3.SI": "Communication",
    "S63.SI": "Industrials",
    "BN4.SI": "Industrials",
    "S51.SI": "Industrials",
    "Z59.SI": "Industrials",
    "O39.SI": "Finance",
    "C09.SI": "Real Estate",
    "F99.SI": "Consumer Defensive"
}
sector_lower_1 = {"Industrials": 0.1, "Communication": 0.1, "Real Estate": 0.05} 
sector_upper_1 = {
    "Consumer Defensive": 0.3,
    "Finance": 0.5
}
variations = []
var1 = [sector_mapper_1, sector_lower_1, sector_upper_1]
variations.append(var1)

sector_mapper_2 = {
    "Z74.SI": "Communication",
    "D05.SI": "Finance",
    "S68.SI": "Finance",
    "C38U.SI": "Real Estate",
    "S58.SI": "Industrials",
    "S08.SI": "Industrials",
    "C6L.SI": "Industrials",
    "CC3.SI": "Communication",
    "S63.SI": "Industrials",
    "BN4.SI": "Industrials",
    "S51.SI": "Industrials",
    "Z59.SI": "Industrials",
    "O39.SI": "Finance",
    "C09.SI": "Real Estate",
    "F99.SI": "Consumer Defensive"
}
sector_lower_2 = {"Industrials": 0.1, "Communication": 0.1, "Real Estate": 0.1} 
sector_upper_2 = {
    "Consumer Defensive": 0.2,
    "Finance": 0.5
}
var2 = [sector_mapper_2, sector_lower_2, sector_upper_2]
variations.append(var2)


### Train Results

In [39]:
for i in range(len(variations)):
    print("Variation " + str(i))
    Engine(stock_list,"2015-01-01", "2022-12-31", variations[i])
    print("\n")

Variation 0
Expected annual return: 7.4%
Annual volatility: 15.6%
Sharpe Ratio: 0.35
OrderedDict([('Z74.SI', 0.1), ('D05.SI', 0.5), ('S68.SI', 0.0), ('C38U.SI', 0.27266), ('S58.SI', 0.0), ('S08.SI', 0.0), ('C6L.SI', 0.0), ('CC3.SI', 0.0), ('S63.SI', 0.12734), ('BN4.SI', 0.0), ('S51.SI', 0.0), ('Z59.SI', 0.0), ('O39.SI', 0.0), ('C09.SI', 0.0), ('F99.SI', 0.0)])
----------------------Weight method----------------------
Portfolio Return: 0.07317355081193644
Portfolio Volatility: 0.16874103919928807
Sharpe Ratio: 0.4336440688001
----------------------Share method----------------------
Total Portfolio Return with Discrete Allocation: Portfolio Value    0.079094
dtype: float64
Portfolio Volatility with Discrete Allocation: Portfolio Value    0.158129
dtype: float64
Sharpe Ratio with Discrete Allocation: Portfolio Value    0.500182
dtype: float64


Variation 1
Expected annual return: 7.4%
Annual volatility: 15.6%
Sharpe Ratio: 0.35
OrderedDict([('Z74.SI', 0.1), ('D05.SI', 0.5), ('S68.SI', 0.0

### Test Results

In [17]:
Engine(stock_list,"2023-01-01", "2023-11-19")

Expected annual return: 17.8%
Annual volatility: 10.4%
Sharpe Ratio: 1.52
Portfolio Return: 0.17715575562433772
Portfolio Volatility: 0.10256261269482336
Sharpe Ratio: 1.7272937083951576
Total Portfolio Return with Discrete Allocation: Portfolio Value    0.177747
dtype: float64
Portfolio Volatility with Discrete Allocation: Portfolio Value    0.099331
dtype: float64
Sharpe Ratio with Discrete Allocation: Portfolio Value    1.789429
dtype: float64
